# Task 1 - Statistical analysis 

In this part of the task, we are asked to plan and implement a statistical analysis to characterize the length of the shortest path in dependence on several of the grid's parameters. 

The relevant parameters are: 
- the grid's size
- the distribution from which cell numbers are generated.